In [1]:
%config Completer.use_jedi = False
# %matplotlib widget



In [4]:
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import pandas as pd
import seaborn as sns
import scipy
from sklearn import metrics
import pybedtools
import coolbox
from coolbox.api import *

from joblib.externals.loky import set_loky_pickler
from joblib import parallel_backend
from joblib import Parallel, delayed
from joblib import wrap_non_picklable_objects

In [ ]:
!cut -f2,3,4 NC_016776R0415-CWH.bedgraph

In [15]:
primary=pd.read_excel('Data Raw - Gut Microbiome Cohort Project Database - 300 Cohort v3.0_280921.xlsx',index_col=0,sheet_name='Primary Data')
diet=pd.read_excel('Data Raw - Gut Microbiome Cohort Project Database - 300 Cohort v3.0_280921.xlsx',index_col=0,sheet_name='Diet Data')
blood_stool=pd.read_excel('Data Raw - Gut Microbiome Cohort Project Database - 300 Cohort v3.0_280921.xlsx',index_col=0,sheet_name='blood and stool biomarkers')
secondary=pd.read_excel('Data Raw - Gut Microbiome Cohort Project Database - 300 Cohort v3.0_280921.xlsx',index_col=0,sheet_name='Secondary Data')
MRI=pd.read_excel('Data Raw - Gut Microbiome Cohort Project Database - 300 Cohort v3.0_280921.xlsx',index_col=0,sheet_name='MRI scores')

In [469]:
oric=pd.read_csv('data/doric10/tubic_bacteria.csv') #remove 'note' and 'nt*' from file

In [470]:
oric['oric_start']=oric['Location of replication origin'].str.split('.').str[0].str.split(',').str[0].str.split(';').str[0].tolist()
oric['oric_end']=oric['Location of replication origin'].str.split('.').str[2].str.split(',').str[0].str.split(';').str[0].tolist()
oric['gene_start']=oric['Location of replication genes'].str.split('.').str[0].str.split(',').str[0].str.split(';').str[0].tolist()
oric['gene_end']=oric['Location of replication genes'].str.split('.').str[2].str.split(',').str[0].str.split(';').str[0].tolist()

In [471]:
oric['OriC AT content']=np.round(oric['OriC AT content']*100).astype(int)
oric['orgA']=oric['Organism'].str.split(' ').str[0].tolist()
oric['orgB']=oric['Organism'].str.split(' ').str[1].tolist()
oric['Organism']=oric['orgA']+'_'+oric['orgB']


In [472]:
oric[['oric_start','oric_end','gene_start','gene_end']]=oric[['oric_start','oric_end','gene_start','gene_end']].replace('-','1')
oric=oric.fillna('1')

In [473]:
oric[['gene_start']]=oric[['gene_start']].astype(int)
oric[['gene_end']]=oric[['gene_end']].astype(int)
oric[['oric_start']]=oric[['oric_start']].astype(int)
oric[['oric_end']]=oric[['oric_end']].astype(int)

In [474]:
oric['end']=np.amax(oric[['gene_start','oric_end','gene_start','gene_end']],1)
oric['start']=np.amin(oric[['gene_start','oric_end','gene_start','gene_end']],1)

In [476]:
# oric['start']=np.amin(oric,1)#
# oric['end']=np.amax(oric,1)
# oric['index']=1
oric[['Refseq','start','end','OriC AT content','Organism']]

,Refseq,start,end,OriC AT content,Organism
0,NC_000908.1,577268,579223,72,Mycoplasma_genitalium
1,NC_000919.1,4,1640,67,Treponema_pallidum
2,NC_000964.1,410,1938,63,Bacillus_subtilis
3,NC_000964.1,409,1750,65,Bacillus_subtilis
4,NC_001263.1,1903,3268,52,Deinococcus_radiodurans
...,...,...,...,...,...
9664,NZ_CP024968,1,1556,81,Mesoplasma_coleopterae
9665,NZ_CP024968,1,800407,79,Mesoplasma_coleopterae
9666,NZ_CP024969,1,1559,82,Mesoplasma_tabanidae
9667,NZ_CP024969,1,846907,78,Mesoplasma_tabanidae


In [477]:
oric[['Refseq','start','end','OriC AT content','Organism']].to_csv('OriC_db.bed',sep='\t',header=True,index=False)

In [45]:
traces= glob.glob('data/meta/*bedgraph')
jac=traces[7]
ori=pd.read_csv('OriC_db.bed',sep='\t')

In [ ]:
zzz=pd.read_csv(jac,sep='\t',names=['tmp','start','end','ratio'])
tmp=ori[ori['Refseq'].str.contains('NC_'+jac.split('/')[2].split('_')[1])]
plt.figure(figsize=(20, 5))

plt.bar(data=zzz[(zzz['start']>3000000)&(zzz['end']<3500000)],x='start',height='ratio',label='align',color='blue')#][zzz['start']=='1665929']
plt.bar(data=tmp,x='start',height='OriC AT content',label='OriC start',color='orange')
plt.bar(data=tmp,x='end',height='OriC AT content',label='OriC end',color='orange')

plt.axes([.2, .6, .6, .2])
plt.bar(data=zzz[(zzz['start']>np.int(np.round(tmp['start'].item(),-1))-100) &(zzz['end']<np.int(np.round(tmp['end'].item(),-1))+100)],x='start',height='ratio',label='align',color='blue')#][zzz['start']=='1665929']
plt.bar(data=tmp,x='start',height='OriC AT content',label='OriC start',color='orange')
plt.bar(data=tmp,x='end',height='OriC AT content',label='OriC end',color='orange')


plt.xlabel("OriC ranges from "+str(tmp['start'].item())+" to "+str(tmp['end'].item())+" for "
        +str(tmp['Organism'].item())+" in "+str(os.path.basename(jac).split('_')[2].split('.')[0])+
           "max= "+np.max(zzz['ratio'])+" at "+str(zzz[zzz['ratio']==np.max(zzz['ratio'])][['start']].index.values)+"; median= "+np.median(zzz['ratio']),fontsize=14)
 

In [ ]:
traces= glob.glob('data/meta/*bedgraph')
ori=pd.read_csv('OriC_db.bed',sep='\t')

for jac in traces:
    zzz=pd.read_csv(jac,sep='\t',names=['tmp','start','end','ratio'])
    tmp=ori[ori['Refseq'].str.contains('NC_'+jac.split('/')[2].split('_')[1])]
    
    plt.figure(figsize=(20, 5))
    try:
        print('running: '+str(tmp['Organism'].item())+" in "+str(os.path.basename(jac).split('_')[2].split('.')[0]))
        ss=np.int(np.round(tmp['start'].item(),-1))
        tt=np.int(np.round(tmp['end'].item(),-1))
        plt.bar(data=zzz[(zzz['start']>ss-250000)&(zzz['end']<tt+250000)],x='start',height='ratio',label='align',color='blue')#][zzz['start']=='1665929']
        plt.bar(data=tmp,x='start',height='OriC AT content',label='OriC start',color='orange')
        plt.bar(data=tmp,x='end',height='OriC AT content',label='OriC end',color='orange')

        plt.axes([.2, .6, .6, .2])
        plt.bar(data=zzz[(zzz['start']>ss-100) &(zzz['end']<tt+100)],x='start',height='ratio',label='align',color='blue')#][zzz['start']=='1665929']
        plt.bar(data=tmp,x='start',height='OriC AT content',label='OriC start',color='orange')
        plt.bar(data=tmp,x='end',height='OriC AT content',label='OriC end',color='orange')

        plt.xlabel("OriC ranges from "+str(tmp['start'].item())+" to "+str(tmp['end'].item())+" for "
            +str(tmp['Organism'].item())+" in "+str(os.path.basename(jac).split('_')[2].split('.')[0])+"max= "+
                   np.max(zzz['ratio'])+" between "+str(np.int(zzz[zzz['ratio']==np.max(zzz['ratio'])][['start']].values))+" and "+str(np.int(zzz[zzz['ratio']==np.max(zzz['ratio'])][['end']].values))+"; median= "+np.median(zzz['ratio']),fontsize=14)
        plt.savefig('img/oric/'+str(os.path.basename(jac).split('.')[0])+'.png',dpi=300,bbox_inches = "tight")

    except TypeError:
        pass
    except ValueError:
        pass

In [79]:
table=pd.DataFrame()
traces= glob.glob('data/meta/*bedgraph')
ori=pd.read_csv('OriC_db.bed',sep='\t')

for jac in traces:
    zzz=pd.read_csv(jac,sep='\t',names=['tmp','start','end','ratio'])
    tmp=ori[ori['Refseq'].str.contains('NC_'+jac.split('/')[2].split('_')[1])]
    try:
        ss=np.int(np.round(tmp['start'].item(),-1))
        tt=np.int(np.round(tmp['end'].item(),-1))
        tmpd=zzz[(zzz['start']>ss) &(zzz['end']<tt)]

        tmp['mean']=np.mean(zzz['ratio'])
        tmp['ori_mean']=np.mean(tmpd['ratio'])

        tmp['median']=np.median(zzz['ratio'])
        tmp['ori_median']=np.median(tmpd['ratio'])
        table=pd.concat([table,tmp],axis=0)
    except ValueError:
        pass

In [81]:
# pd.DataFrame(table[:,0,:])
table.to_csv('data/oriC_calc.txt',sep='\t')

In [85]:
np.sum(table['mean']>table['ori_mean'])/len(table)

0.34782608695652173

In [86]:
np.sum(table['median']>table['ori_median'])/len(table)

0.15942028985507245